In [1]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
from time import time
from datetime import datetime, timedelta

In [2]:
start_time = time()

energy_meters_df = pd.read_excel('data/VIS Målere.xlsx')
# Remove leading and trailing whitespace in cells with value of type string
energy_meters_df = energy_meters_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# parse_dates=['Unnamed: 0']

raw_esave_tables_dict = pd.read_excel('data/EsaveExport_Trondheim Kommune_Trondheim_10121314.xls', decimal=',', sheet_name=None)
raw_esave_tables_list = raw_esave_tables_dict.values()
for table in raw_esave_tables_list:
    table.rename(columns={'Unnamed: 0': 'datetime'}, inplace=True)
    table['datetime'] = pd.to_datetime(table['datetime'], dayfirst=True)
    table.set_index('datetime', inplace=True)
    table.sort_index()
raw_esave_table = pd.concat(raw_esave_tables_list, axis=1, ignore_index=False)

stop_time = time()
elapsed_time = stop_time - start_time
print(f'Used {elapsed_time:.2f} seconds to read xls files')


In [ ]:
energy_meters_df

In [ ]:
raw_esave_table

Types of sensors and coresponding unit of measurement:  
'Ikke valgt(Ikke valgt)','Elkjel(kWh)', 'Temperatur(°C)', 'Ikke valgt(°C)', 'Fjernvarme(kWh)', 'Ikke valgt(m/s)', 'Vann(m3)', 'Ikke valgt(kWh)', 'Ikke valgt(timer)', 'Fastkraft(kWh)', 'Virkningsgrad(nan)', 'Kjøling(kWh)', 'Varme(kWh)', 'Virkningsgrad(Ikke valgt)', 'Graddager(grdg)'

In [ ]:
start_time = time()

buildings = {}
current_building = None
sensor_types = ['Fastkraft', 'Fjernvarme', 'Varme', 'Elkjel', 'Kjøling']
for _, row in energy_meters_df.iterrows():
    # if current row is a building not a sensor create a new dict to store all sensors for that building
    if row['Objekt'] == 'Bygg':
        current_building = {}
        for sensor_type in sensor_types:
            current_building[sensor_type] = {}
        buildings[row['Navn']] = current_building
        continue
    
    sensor_type = row['Type']
    if sensor_type in sensor_types:
        name = row['Navn']
        sensor_id = row['Formel']
        current_building[sensor_type][name] = sensor_id


stop_time = time()
elapsed_time = stop_time - start_time
print(f'Used {elapsed_time:.2f} seconds to parse')

In [ ]:
# Create a dictionary that contains a dataframe for each building that have column for each sensor type (e.g. Fjernvarme or Fastkraft)
building_dfs = {}
for building_name, sensor_type_dict in buildings.items():

    sensor_type_series = {}
    for sensor_type, sensor_dict in sensor_type_dict.items():
        for sensor_id in sensor_dict.values():
            if sensor_id in raw_esave_table.columns:
                sensor_series = raw_esave_table[sensor_id]
                if not sensor_type in sensor_type_series:
                    sensor_type_series[sensor_type] = sensor_series
                else:
                    sensor_type_series[sensor_type] = sensor_type_series[sensor_type] + sensor_series
    if sensor_type_series:
        building_dfs[building_name] = pd.DataFrame(sensor_type_series)
        building_dfs[building_name].sort_index()

# Create a column in each building dataframe that is the total energy consumption, i.e. the sum of all other columns.
for building_df in building_dfs.values():
    building_df['Totalt'] = building_df[list(building_df.columns)].sum(axis=1)


In [ ]:
building_dfs['Trondheim Kommune - Voldsminde barnehage'].loc[datetime(2018,1,):datetime(2019,1,1)]
building_dfs['Trondheim Kommune - Voldsminde barnehage'].loc['2018-1-1':]

In [ ]:
building_dfs['Trondheim Kommune - Voldsminde barnehage'].plot()
plt.show()

In [ ]:
# Create new data frames for each building that have values for each day or week instead of hourly
building_daily_dfs = {}
building_weekly_dfs = {}
for building_name, building_df in building_dfs.items():
    building_daily_dfs[building_name] = building_df.resample('D').sum()
    building_weekly_dfs[building_name] = building_df.resample('W-MON').sum()


In [ ]:
building_daily_dfs['Trondheim Kommune - Voldsminde barnehage'].plot()
plt.show()
building_weekly_dfs['Trondheim Kommune - Voldsminde barnehage'].plot()
plt.show()
